## Spark 101 Exercises
### Corey Solitaire
`11.24.2020`

In [1]:
import pyspark
import pandas as pd
import numpy as np

from pydataset import data
from vega_datasets import data

from pyspark.sql.functions import col, expr, concat, sum, avg, min, max, count, mean, round
from pyspark.sql.functions import lit, regexp_extract, regexp_replace, when,asc, desc, month, year, quarter  

***
## Method to create spark session

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

***
## Reference Material

In [3]:
pd_v_spark = pd.DataFrame([['pd.read_csv("myfile.csv")', 
                            'spark.read.load("myfile.csv", format = "csv", sep = ",")'], 
                           ['pd.read_json("myfile.json")', 
                            'spark.read.load("myfile.json", format = "json") OR spark.read.json("myfile.json")']], 
                          index = ['csv', 'json'], 
                          columns = ['pandas', 'spark'])

pd_v_spark = pd_v_spark.append(pd.DataFrame([['pd_df.head()', 'sp_df.show(), .head(), .take()'],
                                             ['pd_df.head(1)', 'sp_df.first()'],
                                             ['pd_df.describe()', 'sp_df.describe()'],
                                             ['pd_df.columns', 'sp_df.columns'],
                                             ['len(pd_df)', 'sp_df.count()'],
                                             ['len(pd_df.drop_duplicates())', 'sp_df.distinct().count()'],
                                             ['pd_df.info()', 'sp_df.printSchema()']
                                            ],
                                            index = ['1st n rows', '1st row','summary statistics', 
                                                     'column names', '# rows', '# distinct rows', 
                                                     'df schema info'], 
                                            columns = ['pandas', 'spark']))

pd_v_spark = pd_v_spark.append(pd.DataFrame([['pd_df[["col1", "col2"]]', 
                                              'sp_df.select(sp_df.col1, sp_df.col2)']
                                            ],
                                            index = ['select columns'], 
                                            columns = ['pandas', 'spark']))

pd_v_spark = pd_v_spark.append(pd.DataFrame([['pd_df[pd_df.c1 > 0]', 'sp_df.filter(df.c1 > 0), sp_df.where(df.c1 > 0)'],
                                            ],
                                            index = ['conditional filtering'], 
                                            columns = ['pandas', 'spark']))

pd_v_spark = pd_v_spark.append(pd.DataFrame([['np.where(pd_df.c1.array > 0, "positive")', 
                                              'sp_df.select(df.c1, when(df.c1 > 0, "positive").alias("number_sign"))'],
                                            ],
                                            index = ['conditional assigning'], 
                                            columns = ['pandas', 'spark']))

pd_v_spark = pd_v_spark.append(pd.DataFrame([['np.where(pd_df.c1.array > 0, "pos", "neg")', 
                                              'sp_df.select(df.c1, when(df.c1 > 0, "pos").otherwise("neg").alias("number_sign"))'],
                                            ],
                                            index = ['conditional assigning with else'], 
                                            columns = ['pandas', 'spark']))


pd_v_spark = pd_v_spark.append(pd.DataFrame([['pd_df.sort_values(by=["c1"])', 
                                              'sp_df.sort(sp_df.c1)'],
                                             ['pd_df.sort_values(by=["c1","c2"])',
                                              'sp_df.sort(sp_df.c1, sp_df.c2)'],
                                             ['pd_df.sort_values(by=["c1","c2"], ascending=[False, True])',
                                              'sp_df.sort(sp_df.c1.desc(), sp_df.c2)'],
                                             ['pd_df.sort_values(by=["c1","c2"], ascending=False)', 
                                              'sp_df.sort(desc("c1"), desc("c2")) OR sp_df.sort(col("c1").desc(), col("c2").desc())']
                                            ],
                                            index = ['sort 1 col asc', 'sort 2+ cols asc', 'sort 2+ cols desc/asc', 'sort 2+ cols desc'], 
                                            columns = ['pandas', 'spark']))
pd_v_spark

,pandas,spark
csv,"pd.read_csv(""myfile.csv"")","spark.read.load(""myfile.csv"", format = ""csv"", sep = "","")"
json,"pd.read_json(""myfile.json"")","spark.read.load(""myfile.json"", format = ""json"") OR spark.read.json(""myfile.json"")"
1st n rows,pd_df.head(),"sp_df.show(), .head(), .take()"
1st row,pd_df.head(1),sp_df.first()
summary statistics,pd_df.describe(),sp_df.describe()
column names,pd_df.columns,sp_df.columns
# rows,len(pd_df),sp_df.count()
# distinct rows,len(pd_df.drop_duplicates()),sp_df.distinct().count()
df schema info,pd_df.info(),sp_df.printSchema()
select columns,"pd_df[[""col1"", ""col2""]]","sp_df.select(sp_df.col1, sp_df.col2)"


***
## 1. Create a spark data frame that contains your favorite programming languages.

### -The name of the column should be language

In [4]:
# Create Dataframe
lang_df = pd.DataFrame([['yes', 'no', 'yes', 'no'], 
                        ['yes', 'yes', 'yes', 'yes']], 
                        index = ['csv', 'json'], 
                        columns = ['python', 'javascript', 'r', 'java'])

# to display and see all text in dataframe
pd.set_option('display.max_colwidth', 10000)


lang_df

,python,javascript,r,java
csv,yes,no,yes,no
json,yes,yes,yes,yes


In [5]:
# Convert to spark
sp_df = spark.createDataFrame(lang_df)
sp_df.show()

+------+----------+---+----+
|python|javascript|  r|java|
+------+----------+---+----+
|   yes|        no|yes|  no|
|   yes|       yes|yes| yes|
+------+----------+---+----+



### -View the schema of the dataframe

In [6]:
sp_df.printSchema()

root
 |-- python: string (nullable = true)
 |-- javascript: string (nullable = true)
 |-- r: string (nullable = true)
 |-- java: string (nullable = true)



### -Output the shape of the dataframe

In [7]:
print((sp_df.count(), len(sp_df.columns)))

(2, 4)


### -Show the first 5 records in the dataframe

In [8]:
sp_df.show(5)

+------+----------+---+----+
|python|javascript|  r|java|
+------+----------+---+----+
|   yes|        no|yes|  no|
|   yes|       yes|yes| yes|
+------+----------+---+----+



***
## 2. Load the mpg dataset as a spark dataframe.

In [11]:
mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



### - A. Create 1 column of output that contains a message like the one below:
   `The 1999 audi a4 has a 4 cylinder engine.`

In [35]:
mpg.select(concat(lit('The '), mpg.year, lit(' '), mpg['manufacturer'], lit(' '), mpg.model, lit(' has a '), mpg.cyl, lit(" cylinder"), lit(' engine')).alias("Discription")).show(20, False)

+-------------------------------------------------------------+
|Discription                                                  |
+-------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine                     |
|The 1999 audi a4 has a 4 cylinder engine                     |
|The 2008 audi a4 has a 4 cylinder engine                     |
|The 2008 audi a4 has a 4 cylinder engine                     |
|The 1999 audi a4 has a 6 cylinder engine                     |
|The 1999 audi a4 has a 6 cylinder engine                     |
|The 2008 audi a4 has a 6 cylinder engine                     |
|The 1999 audi a4 quattro has a 4 cylinder engine             |
|The 1999 audi a4 quattro has a 4 cylinder engine             |
|The 2008 audi a4 quattro has a 4 cylinder engine             |
|The 2008 audi a4 quattro has a 4 cylinder engine             |
|The 1999 audi a4 quattro has a 6 cylinder engine             |
|The 1999 audi a4 quattro has a 6 cylind

### - B. For each vehicle.
        - Transform the trans column so that it only contains either manual or auto.
 

In [59]:
#regex 
mpg.select(regexp_replace("trans", r"\([^)]*\)", "").alias("transonly")).show()

+---------+
|transonly|
+---------+
|     auto|
|   manual|
|   manual|
|     auto|
|     auto|
|   manual|
|     auto|
|   manual|
|     auto|
|   manual|
|     auto|
|     auto|
|   manual|
|     auto|
|   manual|
|     auto|
|     auto|
|     auto|
|     auto|
|     auto|
+---------+
only showing top 20 rows



In [60]:
# not regex
mpg = mpg.withColumn("trans", when(mpg.trans.startswith("a"), "auto").otherwise("manual")).show(6)

+------------+-----+-----+----+---+------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl| trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto|  f| 16| 26|  p|compact|
|        audi|   a4|  2.8|1999|  6|manual|  f| 18| 26|  p|compact|
+------------+-----+-----+----+---+------+---+---+---+---+-------+
only showing top 6 rows



***
## 3. Load the tips dataset as a spark dataframe.

In [62]:
tips = spark.createDataFrame(data("tips"))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



### - A. What percentage of observations are smokers?

In [88]:
tips.groupby(tips.smoker).agg(count(tips.smoker)).show()


tips.groupBy('smoker').agg(round(count(tips.smoker)/ tips.count(),2)).show()

+------+-------------+
|smoker|count(smoker)|
+------+-------------+
|    No|          151|
|   Yes|           93|
+------+-------------+

+------+-------------------------------+
|smoker|round((count(smoker) / 244), 2)|
+------+-------------------------------+
|    No|                           0.62|
|   Yes|                           0.38|
+------+-------------------------------+



### - B. Create a column that contains the tip percentage

In [97]:
tips.select(tips.tip / tips.total_bill).show()

+-------------------+
| (tip / total_bill)|
+-------------------+
|0.05944673337257211|
|0.16054158607350097|
|0.16658733936220846|
| 0.1397804054054054|
|0.14680764538430255|
|0.18623962040332148|
|0.22805017103762829|
|0.11607142857142858|
|0.13031914893617022|
| 0.2185385656292287|
| 0.1665043816942551|
|0.14180374361883155|
|0.10181582360570687|
|0.16277807921866522|
|0.20364126770060686|
|0.18164967562557924|
| 0.1616650532429816|
|0.22774708410067526|
|0.20624631703005306|
|0.16222760290556903|
+-------------------+
only showing top 20 rows



In [99]:
tips = tips.withColumn('tip_percentage', expr('Round((tip/total_bill) * 100)'))
tips.show()

+----------+----+------+------+---+------+----+--------------+
|total_bill| tip|   sex|smoker|day|  time|size|tip_percentage|
+----------+----+------+------+---+------+----+--------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|           6.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|          16.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|          17.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|          14.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|          15.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|          19.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|          23.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|          12.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|          13.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|          22.0|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|          17.0|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|          14.0|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        

### - C. Calculate the average tip percentage for each combination of sex and smoker.

In [90]:
tips.groupBy(tips.smoker, tips.sex).agg(round(avg(tips.tip/tips.total_bill * 100), 2)).show()

+------+------+-----------------------------------------+
|smoker|   sex|round(avg(((tip / total_bill) * 100)), 2)|
+------+------+-----------------------------------------+
|    No|Female|                                    15.69|
|    No|  Male|                                    16.07|
|   Yes|  Male|                                    15.28|
|   Yes|Female|                                    18.22|
+------+------+-----------------------------------------+



***
## 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

In [131]:
from vega_datasets import data
weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



### -Convert the temperatures to farenheight.

In [132]:
weather = weather.withColumn("temp_max", round(weather.temp_max*9/5 + 32, 2))
weater = weather.withColumn("temp_min", round(weather.temp_min*9/5 + 32, 2))
weather.show()

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|   55.04|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|   51.08|     2.8| 4.5|   rain|
|2012-01-03|          0.8|   53.06|     7.2| 2.3|   rain|
|2012-01-04|         20.3|   53.96|     5.6| 4.7|   rain|
|2012-01-05|          1.3|   48.02|     2.8| 6.1|   rain|
|2012-01-06|          2.5|   39.92|     2.2| 2.2|   rain|
|2012-01-07|          0.0|   44.96|     2.8| 2.3|   rain|
|2012-01-08|          0.0|    50.0|     2.8| 2.0|    sun|
|2012-01-09|          4.3|   48.92|     5.0| 3.4|   rain|
|2012-01-10|          1.0|   42.98|     0.6| 3.4|   rain|
|2012-01-11|          0.0|   42.98|    -1.1| 5.1|    sun|
|2012-01-12|          0.0|   42.98|    -1.7| 1.9|    sun|
|2012-01-13|          0.0|    41.0|    -2.8| 1.3|    sun|
|2012-01-14|          4.1|   39.92|     0.6| 5.3|   snow|
|2012-01-15|  

### -Which month has the most rain, on average?

In [133]:
(
    weather.withColumn("month", month("date"))
    .groupBy("month")
    .agg(sum("precipitation").alias("total_rainfall"))
    .sort("month")
    .show()
)


+-----+------------------+
|month|    total_rainfall|
+-----+------------------+
|    1|465.99999999999994|
|    2|             422.0|
|    3|             606.2|
|    4|             375.4|
|    5|             207.5|
|    6|             132.9|
|    7|              48.2|
|    8|             163.7|
|    9|235.49999999999997|
|   10|             503.4|
|   11|             642.5|
|   12| 622.7000000000002|
+-----+------------------+



### -Which year was the windiest?

In [134]:
(
    weather.withColumn("year", month("date"))
    .groupBy("year")
    .agg(sum("wind").alias("total_wind"))
    .sort("year")
    .show()
)

+----+------------------+
|year|        total_wind|
+----+------------------+
|   1|             389.2|
|   2|             427.9|
|   3|443.90000000000003|
|   4|             422.9|
|   5|             386.9|
|   6| 375.7000000000001|
|   7|             361.0|
|   8|             341.1|
|   9|             355.6|
|  10|             364.5|
|  11|             417.9|
|  12|448.69999999999993|
+----+------------------+



### -What is the most frequent type of weather in January?

In [143]:
(
    weather.filter(month("date") == 1)
    .groupBy("weather")
    .agg(count("weather"))
    .show()
)

+-------+--------------+
|weather|count(weather)|
+-------+--------------+
|    fog|            38|
|drizzle|            10|
|   rain|            35|
|    sun|            33|
|   snow|             8|
+-------+--------------+



### -What is the average high and low tempurature on sunny days in July in 2013 and 2014?

In [160]:
(
    weather.filter(month("date") == 7)
    .filter(year("date") == 2013 & 2014)
    .filter(weather.weather == 'sun')
    .agg(mean("temp_min"))
    .show()
)

(
    weather.filter(month("date") == 7)
    .filter(year("date") == 2013 % 2014)
    .filter(weather.weather == 'sun')
    .agg(mean("temp_max"))
    .show()
)

+------------------+
|     avg(temp_min)|
+------------------+
|13.183333333333335|
+------------------+

+-----------------+
|    avg(temp_max)|
+-----------------+
|79.85333333333334|
+-----------------+



### -What percentage of days were rainy in q3 of 2015?

### -For each year, find what percentage of days it rained (had non-zero precipitation).